In [1]:
from pathlib import Path
from IPython.display import Audio

sample = Path(
    'samples/dt01335-95baa5807ed1e7e35260ce3b8a1a4a2e-55-001-vad.wav'
)


In [2]:
import librosa as R
import soundfile as sf

In [4]:
wav, sr = R.load(sample, sr=None)
wav.shape, sr

((59400,), 16000)

In [5]:
wav.min(), wav.max()

(-0.34231567, 0.2192688)

In [9]:
sf.write('sample.wav', wav[:16000*3], sr, format='WAV', subtype='PCM_16')

In [3]:
import pandas as pd
dus = Path('explicit-audios-duration.csv')

dataset = pd.read_csv(dus, header=None, names=['fpath', 'duration'])
dataset.head()

,fpath,duration
0,explicit-audios/xiaomi-vaded-confuse/mi00353-a...,4.168125
1,explicit-audios/xiaomi-vaded-confuse/mi00334-7...,0.590625
2,explicit-audios/xiaomi-vaded-confuse/mi00406-6...,1.046250
3,explicit-audios/xiaomi-vaded-confuse/mi00007-2...,63.770625
4,explicit-audios/xiaomi-vaded-confuse/mi00130-3...,0.725625


In [16]:
longs = dataset[dataset.duration>4]
longs.shape

(23402, 2)

In [5]:
longs.head()

,fpath,duration
3,explicit-audios/xiaomi-vaded-confuse/mi00007-2...,63.770625
6,explicit-audios/xiaomi-vaded-confuse/mi00037-9...,10.479375
11,explicit-audios/xiaomi-vaded-confuse/mi00226-e...,8.235000
18,explicit-audios/xiaomi-vaded-confuse/mi00145-a...,88.678125
21,explicit-audios/xiaomi-vaded-confuse/mi00346-c...,41.833125


In [6]:
from tqdm.auto import tqdm

In [17]:
to = Path('explicit-audios-long')

for row in tqdm(longs.itertuples()):
    fpath = Path(row.fpath)
    if fpath.exists():
        topath = to / fpath
        topath.parent.mkdir(exist_ok=True, parents=True)
        fpath.rename(topath)
    

0it [00:00, ?it/s]

In [18]:
def split_wav(wav, sr, du:float=3, step:float =2.5):
    du = int(sr*du)
    step = int(sr*step)
    wavs = []
    for i in range(0, len(wav), step):
        wavs.append(wav[i:i+du])
    if wavs[-1].shape[0] < du:
        wavs.pop()
        wavs.append(wav[-du:])
    return wavs


In [19]:
tox = Path('explicit-audios-long-splitted')
wavs = list(to.rglob('*.wav'))

for wpath in tqdm(wavs):
    wav, sr = R.load(wpath, sr=None)
    if Path(tox, wpath.with_name(f'{wpath.stem}-0.wav')).exists():
        continue
    wavs = split_wav(wav, sr, du=3, step=2.5)
    for i, subwav in enumerate(wavs):
        fpath = tox / wpath.with_name(f'{wpath.stem}-{i}.wav')
        fpath.parent.mkdir(parents=True, exist_ok = True)
        sf.write(fpath, subwav, sr, format='WAV', subtype='PCM_16')

  0%|          | 0/23402 [00:00<?, ?it/s]